In [1]:
import sys
sys.path.append('~/GPy/GPy') #change to where your GPy is installed or remove if in path
import GPy
import numpy as np
import matplotlib.pyplot as plt
import time
%matplotlib inline

#make some simulated data

N = 6 #number of data points
S = 8  #number of time series per data point
Tavg = 20 #average number of time points per time series (most of these will be unknown)
C = 3  #number of clusters

#actual latent function
def lat_fn(t,s,c): #time, series, cluster
    return np.sin(1.0*(1.0+t)*(1.0+c)/(10.0+c)+c)**(c+1.0)+t*(0.01*c*s)+c*0.2+1-s*0.2

data = []
inputs = []
dataA = []
inputsA = []
groundtruth = []

offsets = np.random.randn(N)*0 #no time offsets

for p in range(N):
    T = np.random.randint(Tavg-5,Tavg+5)
    persondata = np.zeros([S,T])
    personinputs = np.zeros([T,1])
    indx = 0
    c = np.random.randint(0,C) #pick cluster we're going to put person in
    groundtruth.append(c)
    pt = 0
    for t in range(T):
        personinputs[indx,0] = pt
        indx+=1
        for s in range(S):
            persondata[s,t] = lat_fn(pt+offsets[p],s,c) + np.random.randn(1)*0.5
        pt += np.random.randint(1,10)
    data.append(persondata)
    inputs.append(personinputs)

In [3]:
starttime = time.time()
#active = GPy.util.cluster_with_offset.cluster(data,inputs)

active = GPy.util.threaded_cluster_with_offset.cluster(data,inputs)
endtime = time.time()
print "TOTAL TIME %0.4f" % (endtime-starttime)

TOTAL TIME 3.1169


 /home/lionfish/GPy/GPy/core/gp.py:86: UserWarning:Your kernel has a different input dimension 1 then the given X dimension 2. Be very sure this is what you want and you have not forgotten to set the right input dimenion in your kernel
